# Building Datasets
Goal: Build a dataset that can give me the average price per squarefoot for different types of properties.

## 1. Import Libraries

In [2]:
import pandas as pd

## 2. Load the Dataset

In [3]:
df = pd.read_csv('data/NY-House-Dataset.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4801 entries, 0 to 4800
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   BROKERTITLE                  4801 non-null   object 
 1   TYPE                         4801 non-null   object 
 2   PRICE                        4801 non-null   int64  
 3   BEDS                         4801 non-null   int64  
 4   BATH                         4801 non-null   float64
 5   PROPERTYSQFT                 4801 non-null   float64
 6   ADDRESS                      4801 non-null   object 
 7   STATE                        4801 non-null   object 
 8   MAIN_ADDRESS                 4801 non-null   object 
 9   ADMINISTRATIVE_AREA_LEVEL_2  4801 non-null   object 
 10  LOCALITY                     4801 non-null   object 
 11  SUBLOCALITY                  4801 non-null   object 
 12  STREET_NAME                  4801 non-null   object 
 13  LONG_NAME         

## 3. Modifying Columns

### Dropping Columns

In [4]:
# Drop specific column
df.drop("ADMINISTRATIVE_AREA_LEVEL_2", axis=1, inplace=True)

In [5]:
# Drop all except x
# PROPERTYSQFT, PRICE, and TYPE used in class
# BROKERTITLE, BEDS, BATH, SUBLOCALITY, LOCALITY used in lab/homework
df = df[["PROPERTYSQFT", "PRICE", "TYPE", "BROKERTITLE", "BEDS", "BATH", "SUBLOCALITY", "LOCALITY"]]

### Adding Columns

In [6]:
# Calculate $/sqft
df["PERSQFT"] = df["PRICE"] / df["PROPERTYSQFT"]

## 4. Sorting

In [7]:
df.sort_values("PERSQFT", ascending=False)

,PROPERTYSQFT,PRICE,TYPE,BROKERTITLE,BEDS,BATH,SUBLOCALITY,LOCALITY,PERSQFT
304,10000.000000,2147483647,House for sale,Brokered by ANNE LOPA REAL ESTATE,7,6.000000,Richmond County,New York,214748.364700
1075,2184.207862,60000000,Co-op for sale,Brokered by COMPASS,8,8.000000,New York,New York County,27469.913026
1453,2184.207862,48000000,Co-op for sale,Brokered by Corcoran East Side,5,2.373861,New York,New York County,21975.930421
3388,2184.207862,45000000,Co-op for sale,Brokered by Sotheby's International Realty - E...,5,2.373861,New York,New York County,20602.434770
3571,2184.207862,44500000,Co-op for sale,Brokered by Sotheby's International Realty - E...,5,6.000000,New York,New York County,20373.518828
...,...,...,...,...,...,...,...,...,...
318,2184.207862,99000,Condo for sale,Brokered by 5 Boro Realty Corp,1,1.000000,Brooklyn,Kings County,45.325356
4003,2184.207862,75000,Land for sale,Brokered by Prestige Homes Ny Inc,3,2.373861,New York,United States,34.337391
360,2184.207862,5800,Land for sale,Brokered by Century 21 Realty First,3,2.373861,Brooklyn,Kings County,2.655425
310,2184.207862,3225,For sale,Brokered by Living NY - Main Office,3,1.000000,New York,New York County,1.476508


## 5. Removing Rows

In [8]:
# Remove all rows we don't want to keep
keep = ['Condo for sale', 'House for sale', 'Townhouse for sale',
       'Co-op for sale', 'Multi-family home for sale','Mobile house for sale']
filtered_df = df.loc[df['TYPE'].isin(keep)]

## 6. Map
The same as in map, filter, reduce. Allows us to modify every value in a column.

Could use a separate function or a lambda expression.

In [9]:
# Removes the " for sale" if it exists
def remove_for_sale(word):
    index = word.find(" for sale")
    if index != -1:
        return word[:index]
    return word

filtered_df["TYPE"] = filtered_df["TYPE"].map(remove_for_sale)


/var/folders/jf/hz97w2_s6sv3h8hxx1mhrzqr0000gn/T/ipykernel_21847/937672506.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["TYPE"] = filtered_df["TYPE"].map(remove_for_sale)


## 7. Groupby

In [10]:
filtered_df.groupby("TYPE")[["PERSQFT"]].mean()

,PERSQFT
TYPE,
Co-op,605.463668
Condo,1220.072386
House,835.314814
Mobile house,589.687466
Multi-family home,599.004669
Townhouse,1533.853853


## 8. Homework
### Goal 1: Remove the “Brokered by ” for every row in the BROKERTITLE column.

In [11]:
# Remove brokered by using python's built-in removeprefix function
df["BROKERTITLE"] = df["BROKERTITLE"].map(lambda x:x.removeprefix("Brokered by "))
# Show different value counts for each brokertitle
df["BROKERTITLE"].value_counts()

BROKERTITLE
COMPASS                              456
Douglas Elliman - 575 Madison Ave    110
Brown Harris Stevens                  93
Corcoran East Side                    91
RE MAX Edge                           79
                                    ... 
Gerard R Desgranges                    1
Property Professional Realty           1
Mark Murphy Properties                 1
S Sharf Realty Inc                     1
New York Way Real Estate Corp          1
Name: count, Length: 1036, dtype: int64

### Goal 2: Get the max* square footage in each Locality that ends with “County”. For example, we don’t care about “The Bronx” but we care about “Bronx County”.

In [12]:
# Select columns locality and propertysqft, then filter the localities to contain "COUNTY", group by the locality, and find the highest sqft
df[["LOCALITY", "PROPERTYSQFT"]].query("LOCALITY.str.contains('County')").groupby("LOCALITY").max("PROPERTYSQFT")

,PROPERTYSQFT
LOCALITY,
Bronx County,4000.0
Kings County,18936.0
New York County,8360.0
Queens County,5000.0
Richmond County,5400.0


### Goal 3: Get the average ratio of bedrooms to bathrooms per sublocality and sort it in descending order.

In [15]:
# Calculate bed to bath ratio of every property
df["BEDTOBATH"] = df["BEDS"] / df["BATH"]
# Filter columns to only include sublocality and bedtobath, then group by sublocality and find average
df[["SUBLOCALITY", "BEDTOBATH"]].groupby("SUBLOCALITY").mean().sort_values("BEDTOBATH", ascending=False)

,BEDTOBATH
SUBLOCALITY,
New York,inf
New York County,inf
Brooklyn Heights,3.000000
Snyder Avenue,3.000000
Rego Park,3.000000
Bronx County,1.794739
Kings County,1.666987
Queens County,1.644182
Staten Island,1.566617
